In [2]:
import synapseclient
from synapseclient import File
import synapseutils
import pandas as pd
from IPython.display import display

syn = synapseclient.login()
PROJECTID = 'syn8316993'

Welcome, Larsson Omberg!



# Find the files that were uploaded with a query

In [ ]:
files = synapseutils.query2df(syn.chunkedQuery('select name from file where parentId=="syn8317652"'))
files = map(syn.get, files.id)

# Read in the data

In [ ]:
dfs = [pd.read_csv(f.path) for f in files]
display(dfs[0].head())
display(dfs[1].head())
display(dfs[3].head())

# Convert consumption

In [ ]:
name_translation = {'sweden.csv': 'Sweden', 
                    'germany.csv': 'Germany',
                    'france.csv': 'France',
                    'usa.csv': 'United States'}

merged = []
for f, df in zip(files, dfs):
    cols = list(set(['beer', 'wine', 'vokda']) & set(df.columns))
    df['alcohol'] = df[cols].sum(1)
    df['country'] = name_translation[f.name]
    df.drop(cols, axis=1, inplace=True)
    display(df.head())
    merged.append(df)

df = pd.concat(merged)

### Push the content back to Synapse

In [ ]:
df.to_csv('merged_alcohol_consumption.csv', index=False)
outfile = File('merged_alcohol_consumption.csv', parentId=PROJECTID)
outfile = syn.store(outfile, used=files)

In [ ]:
syn.onweb(outfile)